In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import shutil
import pandas as pd
from pathlib import Path

# ========== CONFIG ==========
dataset_root = "/content/drive/MyDrive/Final Spectrogram/Gray/GraySpectrogram_10_Sec_trim"
output_root = "/content/drive/MyDrive/Final Spectrogram/Gray/Lebel_10_Sec_trim"
output_image_dir = os.path.join(output_root, "all_images")
output_csv_path = os.path.join(output_root, "labels.csv")
os.makedirs(output_image_dir, exist_ok=True)

valves = ["mitral", "aortic", "tricuspid", "pulmonary"]
splits = ["train", "val", "test"]

records = []

# ========== LOOP ทุก Valve ==========
for valve in valves:
    dataset_path = os.path.join(dataset_root, f"dataset_{valve}")
    for split in splits:
        for label_folder in ["Normal", "Abnormal"]:
            label = 0 if label_folder == "Normal" else 1
            image_dir = os.path.join(dataset_path, split, label_folder)
            if not os.path.exists(image_dir): continue

            for fname in os.listdir(image_dir):
                if not fname.endswith(".png"): continue

                src = os.path.join(image_dir, fname)
                new_fname = f"{valve}_{split}_{label}_{fname}"
                dst = os.path.join(output_image_dir, new_fname)
                shutil.copy(src, dst)

                records.append({
                    "filename": new_fname,
                    "valve": valve,
                    "label": label,
                    "split": split
                })

# ========== SAVE CSV ==========
df = pd.DataFrame(records)
df.to_csv(output_csv_path, index=False)
print(f"✅ Saved labels.csv with {len(df)} records")


✅ Saved labels.csv with 1115 records


In [3]:
from torch.utils.data import Dataset
from PIL import Image
import torch

valve_to_idx = {"mitral": 0, "aortic": 1, "tricuspid": 2, "pulmonary": 3}

class HeartValveDataset(Dataset):
    def __init__(self, csv_file, image_dir, split, transform=None):
        import pandas as pd
        self.df = pd.read_csv(csv_file)
        self.df = self.df[self.df["split"] == split].reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(self.image_dir, row["filename"])
        image = Image.open(image_path).convert("L")  # grayscale

        if self.transform:
            image = self.transform(image)

        valve_idx = valve_to_idx[row["valve"]]
        label = torch.tensor([row["label"]], dtype=torch.float32)
        return image, valve_idx, label


In [4]:
import torch.nn as nn
import torch

class MultiValveCNN(nn.Module):
    def __init__(self, num_valves=4):
        super().__init__()
        self.embedding = nn.Embedding(num_valves, 16)  # valve → vector
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),  # image → feature
            nn.Linear(32 * 56 * 56 + 16, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1)  # binary output
        )

    def forward(self, x, valve_idx):
        x_feat = self.cnn(x)                         # [B, 32, 56, 56]
        v_feat = self.embedding(valve_idx)           # [B, 16]
        x_feat = torch.flatten(x_feat, 1)            # [B, flat]
        all_feat = torch.cat((x_feat, v_feat), dim=1)
        out = self.fc(all_feat)
        return out


In [5]:
from torchvision import transforms
from torch.utils.data import DataLoader

# ========== CONFIG ==========
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
CSV_PATH = "/content/drive/MyDrive/Final Spectrogram/Gray/Lebel_10_Sec_trim/labels.csv"
IMAGE_DIR = "/content/drive/MyDrive/Final Spectrogram/Gray/Lebel_10_Sec_trim/all_images"

# ========== TRANSFORM ==========
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # grayscale
])

# ========== LOAD DATA ==========
train_set = HeartValveDataset(CSV_PATH, IMAGE_DIR, split="train", transform=transform)
val_set   = HeartValveDataset(CSV_PATH, IMAGE_DIR, split="val", transform=transform)
test_set  = HeartValveDataset(CSV_PATH, IMAGE_DIR, split="test", transform=transform)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# ========== SETUP ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiValveCNN().to(device)

# ========== LOSS FUNCTION WITHOUT POS_WEIGHT ==========
criterion = nn.BCEWithLogitsLoss()  # ไม่มี pos_weight
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 20

# ========== TRACK METRICS ==========
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

# ========== TRAINING ==========
for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    # Training loop
    for images, valve_idx, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, valve_idx, labels = images.to(device), valve_idx.to(device), labels.to(device)

        outputs = model(images, valve_idx)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)

        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    avg_loss = total_loss / total

    # ✅ เก็บค่าต่อ epoch (Training)
    train_losses.append(avg_loss)
    train_accuracies.append(train_acc)

    print(f"✅ Epoch {epoch+1}: Train Loss = {avg_loss:.4f} | Train Acc = {train_acc:.4f}")

    # ========== VALIDATION ==========
    model.eval()  # Set the model to evaluation mode
    val_loss, val_correct, val_total = 0.0, 0, 0

    with torch.no_grad():  # Disable gradient computation for validation
        for images, valve_idx, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{EPOCHS}"):
            images, valve_idx, labels = images.to(device), valve_idx.to(device), labels.to(device)

            outputs = model(images, valve_idx)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs) > 0.5
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    avg_val_loss = val_loss / val_total

    # ✅ เก็บค่าต่อ epoch (Validation)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)

    print(f"✅ Epoch {epoch+1}: Val Loss = {avg_val_loss:.4f} | Val Acc = {val_acc:.4f}")

    # ========== SAVE MODEL ==========
    model_save_path = f"/content/drive/MyDrive/Final Spectrogram/Final Model/Gray_CNN_Data_Pretrain_model_epoch_{epoch+1}.pt"
    torch.save(model.state_dict(), model_save_path)
    print(f"💾 Saved model at epoch {epoch+1} to {model_save_path}")
